In the program, we have the last digit of sid `7`, so the order no. is `7`,$S1$ is 688 *China Overseas*, initial stock price `12.18`, volatility `43.6%`; $S2$ is 857 *Petrochina*, initial stock price `6.03`, volatility `30.0%`; $S1/S2$ correlation coefficient `0.304`, Group 3.

For Group 3, the $F\%=102.6\%, UB\%=130.0\%, A\%=122.0\%$

All codes below were executed on an Intel Core i5-12500H chip Windows platform with 40G of RAM. For the programs below, about 1.5G available RAM is required.

Programmes are executed on Python 3.10, package of `numpy` is required.

### Q3

#### Product description
HSBC's shark fin structed note with floor, based on stock $S1$

| Norminal amount (NOM)           | HK$100,000                                                   |
| ------------------------------- | ------------------------------------------------------------ |
| Start Date ($D_s$)             | $t$                                                          |
| Initial stock price             | $S_0$                                                        |
| Stock price before maturity     | $S$                                                          |
| Expiry Date ($T$)               | $t+1.25$ year                                                |
| Expiry date stock closing price | $S_M$                                                        |
| Strike ($K$)                    | $S_0 * 100\%$                                                |
| Barrier price ($P_u$)           | $S_0 * UB\%$                                                 |
| Floor                           | $F\%$                                                        |
| Participation rate              | $A\%$                                                        |
| Payoff at maturity              | 1. payoff = $NOM*F\%$, if the barrier event is triggered          OR<br />2. payoff = $NOM*[100\%+A*\max(S_M/S_0-100\%, 0)]$, if the barrier event is not triggered |
| Barrier event                   | $S>P_u$ at any time before maturity or $S_M>P_u$ at maturity |

Continuously compounded interest rate *r = 4.34% p.a.*

$F\%, UB\%$ and $A\%$ should be based on the Group number of stock *S1*. 

### Q3(i)

Calculate the fair price of this product using a Monte Carlo scheme with time steps $N = 250, i.e. \Delta t = T/N = 1/200$ (refer to the discretization scheme in Topic 1-2, slides 37 and 38).  Give the answer with 100000 paths. 

In [1]:
import numpy as np

def calculate_fair_price(S0: float = 12.18, NOM: int = 100_000, 
                         volatility: float = 43.6/100, r: float = 4.34/100, 
                         T: float = 1.25, N: int = 250, dt: float = 1/200,
                         num_paths: int = 100_000, F: float = 102.6/100, 
                         UB: float = 130.0/100, A: float = 122/100) -> tuple:
    """
        Parameters:
            - S0 (float): Initial stock price. Default is 12.18.
            - NOM (int): Nominal amount of the investment. Default is 100,000.
            - volatility (float): Annualized volatility of the stock. Default is 0.436.
            - r (float): Continuously compounded annual interest rate. Default is 0.0434.
            - T (float): Time to maturity in years. Default is 1.25.
            - N (int): Number of time steps in the simulation. Default is 250.
            - dt (float): Time step size, calculated as T/N. Default is 1/200.
            - num_paths (int): Number of Monte Carlo simulation paths. Default is 100,000.
            - F (float): Floor percentage of the nominal amount. Default is 1.026.
            - UB (float): Upper barrier percentage of the initial stock price. Default is 1.30.
            - A (float): Participation rate for the upside potential. Default is 1.22.

        Returns:
            A tuple containing:
                - fair_price (float): The fair price of the structured note.
                - initial_profit (float): The initial profit of the investment bank.
    """
    Pu = S0 * UB
    K = S0 * 100 / 100
    Z = np.random.normal(0, 1, (num_paths, N))
    S = np.zeros((num_paths, N + 1))
    S[:, 0] = S0

    for t in range(1, N + 1):
        S[:, t] = S[:, t - 1] * np.exp((r - volatility ** 2 / 2) * dt + \
                                       volatility * np.sqrt(dt) * Z[:, t - 1])

    # Check if barrier is triggered
    barrier_triggered = np.any(S > Pu, axis=1)
    # Calculate payoff at maturity
    payoff = np.zeros(num_paths)
    for i in range(num_paths):
        if barrier_triggered[i]:
            payoff[i] = NOM * F
        else:
            S_M = S[i, -1]
            payoff[i] = NOM * (1 + A * max(S_M / S0 - 1, 0))
    # Discount payoffs to present value
    discounted_payoff = np.exp(-r * T) * payoff
    fair_price = np.mean(discounted_payoff)

    return fair_price, NOM - fair_price

result = calculate_fair_price()
print(f"The fair price of the product is: HK${result[0]:.2f}")

The fair price of the product is: HK$96677.88


### Q3(ii)

If the product is sold to an investor at a price of 100% of the nominal amount, calculate the initial profit of the investment bank. 

In [2]:
print(f"The initial profit of the investment bank is: HK${result[1]:.2f}")

The initial profit of the investment bank is: HK$3322.12


### Q3(iii)

The profit level of the investment bank depends on the ombination of $F\%$, $UB\%$, and $A\%$. In each part, calculate the initial profit of the investment bank, and give a brief reason to explain the change in the profit level compared to the answer in (ii) above: 

(a) keeping $F\%$ and $A\%$ unchanged, change $UB\%$ to $115\%$.  Also give the answer if $UB\%$ is changed to $135\%$.  

(a).1 UB = 115%

In [3]:
result3a1 = calculate_fair_price(UB=115/100)
print(f"The initial profit of the investment bank is: HK${result3a1[1]:.2f}")

The initial profit of the investment bank is: HK$3443.43


(a).2 UB=135%

In [4]:
result3a2 = calculate_fair_price(UB=135/100)
print(f"The initial profit of the investment bank is: HK${result3a2[1]:.2f}")

The initial profit of the investment bank is: HK$3064.11


(b) keeping $F\%$ and $UB\%$ unchanged, change $A\%$ to $70\%$.  Also give the answer if $A\%$ is changed to $130\%$. 

(b).1 A = 70%

In [5]:
result3b1 = calculate_fair_price(A=70/100)
print(f"The initial profit of the investment bank is: HK${result3b1[1]:.2f}")

The initial profit of the investment bank is: HK$3610.69


(b).2 A=130%

In [6]:
result3b2 = calculate_fair_price(A=130/100)
print(f"The initial profit of the investment bank is: HK${result3b2[1]:.2f}")

The initial profit of the investment bank is: HK$3269.34


(c)  keeping $UB\%$ and $A\%$ unchanged, change $F\%$ to $104.0\%$. 

(c).1 F = 104%

In [7]:
result3c1 = calculate_fair_price(F=104/100)
print(f"The initial profit of the investment bank is: HK${result3c1[1]:.2f}")

The initial profit of the investment bank is: HK$2633.74
